## Smart Location Database

-  Variable Legend:
  https://geodata.epa.gov/arcgis/rest/services/OA/SmartLocationDatabase/MapServer/layers

In [1]:
!pip install fiona
import geopandas as gpd
import fiona
import os
import numpy as np

In [2]:
### ON DRIVE

# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

# Path to your GDB file (update this path)
gdb_path = '/content/drive/MyDrive/Junior/INFO 3130/Final Project/Community - Connie/SLD.gdb'

# Check the layers available in the GDB
layers = fiona.listlayers(gdb_path)

# Read a specific layer (for example, the first one)
layer_name = layers[0]  # you can change this to any layer you want to load
smart_location_df = gpd.read_file(gdb_path, layer=layer_name)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
### ON GITHUB

# Path to your GDB file (update this path)
#gdb_path = 'SLD.gdb'

# Check the layers available in the GDB
#layers = fiona.listlayers(gdb_path)

# Read a specific layer (for example, the first one)
#layer_name = layers[0]  # you can change this to any layer you want to load
#smart_location_df = gpd.read_file(gdb_path, layer=layer_name)

In [4]:
ny_smart_location_df = smart_location_df[smart_location_df['STATEFP'] == '36'].reset_index(drop=True)
ny_smart_location_df.set_index('GEOID20', inplace=True)
ny_smart_location_df.head()

,GEOID10,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,CBSA_Name,CBSA_POP,...,VMT_per_worker,VMT_tot_min,VMT_tot_max,VMT_tot_avg,GHG_per_worker,Annual_GHG,SLC_score,Shape_Length,Shape_Area,geometry
GEOID20,,,,,,,,,,,,,,,,,,,,,
360150102001,360150102001,36,015,010200,1,236,"Elmira-Corning, NY",21300,"Elmira, NY",85740.0,...,25.382988,7.690275,32.722666,19.519636,22.616242,5880.222946,29.320723,36464.422442,4.192845e+07,"MULTIPOLYGON (((1549562.097 2296224.275, 15495..."
360630243032,360630243032,36,063,024303,2,160,"Buffalo-Cheektowaga-Olean, NY",15380,"Buffalo-Cheektowaga, NY",1131570.0,...,23.194375,7.344131,28.629467,17.468909,20.666189,5373.209019,25.534440,22887.021935,3.415362e+07,"MULTIPOLYGON (((1380986.983 2370657.447, 13809..."
360070143012,360070143012,36,007,014301,2,None,None,13780,"Binghamton, NY",243447.0,...,25.739532,9.909722,32.589187,20.926535,22.933923,5962.820031,30.202013,14968.354915,5.837457e+06,"MULTIPOLYGON (((1629494.517 2293876.766, 16294..."
360810259002,360810259002,36,081,025900,2,408,"New York-Newark, NY-NJ-CT-PA",35620,"New York-Newark-Jersey City, NY-NJ-PA",19318471.0,...,3.184032,1.032142,36.832113,11.177436,2.836972,737.612823,93.989129,1016.744396,6.461912e+04,"MULTIPOLYGON (((1833576.398 2184608.481, 18336..."
360810166002,360810166002,36,081,016600,2,408,"New York-Newark, NY-NJ-CT-PA",35620,"New York-Newark-Jersey City, NY-NJ-PA",19318471.0,...,5.948216,1.032142,36.832113,11.177436,5.299860,1377.963642,86.267939,1485.032311,9.137724e+04,"MULTIPOLYGON (((1842194.593 2179155.182, 18422..."




---



## Research Question:
### How does urban design (combination of land use diversity/density & street design) correlate with the accessibility of public transportation?


Variables:


* Urban Design
  * Density / Entropy & Land Use
    * D2B_E8MIX : 8-tier employment entropy (denominator set to observed employment types in the CBG)
    * D1A: Gross residential density (HU/acre) on unprotected land
    * D1B: Gross population density (people/acre) on unprotected land
    * D1C: Gross employment density (jobs/acre) on unprotected land
    * D2R_JOBPOP: Regional Diversity. Standard calculation based on population and total employment: Deviation of CBG ratio of jobs/pop from the regional average ratio of jobs/pop
  * Street Network
    * D5DRI: Regional Centrality Index of Transit
    * NatWalkInd: Walkability Index
    * D3A: Road Network Density
    * D3B: Street intersection density (weighted, auto-oriented intersections eliminated)
* Accessibility of Public Transportation:
  * D5DR: Proportional Accessibility of Regional Destinations by Transit (Employment accessibility expressed as a ratio of total MSA accessibility)

In [5]:
rqs_df = ny_smart_location_df[['D2B_E8MIX', 'D1A', 'D1B', 'D1C', 'D2R_JOBPOP', 'D5DRI', 'NatWalkInd', 'D3A', 'D3B', 'D5DR']]

rqs_df.head()

,D2B_E8MIX,D1A,D1B,D1C,D2R_JOBPOP,D5DRI,NatWalkInd,D3A,D3B,D5DR
GEOID20,,,,,,,,,,
360150102001,0.956472,0.029261,0.078126,0.011395,0.254585,-99999.000000,5.166667,2.049676,1.092412,-99999.000000
360630243032,0.581794,0.075598,0.216960,0.012797,0.111398,-99999.000000,3.166667,2.197988,1.466803,-99999.000000
360070143012,0.802748,0.246363,0.574848,3.218809,0.303057,-99999.000000,8.166667,7.899193,14.398414,-99999.000000
360810259002,0.744217,28.056812,80.037065,57.240906,0.833942,0.506986,17.333333,25.424383,133.630585,0.000283
360810166002,0.560604,19.796932,69.444270,6.288958,0.166082,0.029180,14.333333,29.468186,113.378400,0.000016


In [6]:
## Option 1: Altering Rows based on Constraints
op = rqs_df.copy()

# Enforce constraints for D2B_E8MIX, D5DR, and D5DRI (between 0 and 1)
op['D2B_E8MIX'] = np.clip(op['D2B_E8MIX'], 0, 1)
op['D5DR'] = np.clip(op['D5DR'], 0, 1)
op['D5DRI'] = np.clip(op['D5DRI'], 0, 1)

# Enforce constraints for D1A, D1B, D1C, D2R_JOBPOP, D3A, and D3B (non-negative)
op['D1A'] = op['D1A'].clip(lower=0)
op['D1B'] = op['D1B'].clip(lower=0)
op['D1C'] = op['D1C'].clip(lower=0)
op['D2R_JOBPOP'] = op['D2R_JOBPOP'].clip(lower=0)
op['D3A'] = op['D3A'].clip(lower=0)
op['D3B'] = op['D3B'].clip(lower=0)

# Enforce constraints for NatWalkInd (between 0 and 100)
op['NatWalkInd'] = np.clip(op['NatWalkInd'], 0, 100)
op

,D2B_E8MIX,D1A,D1B,D1C,D2R_JOBPOP,D5DRI,NatWalkInd,D3A,D3B,D5DR
GEOID20,,,,,,,,,,
360150102001,0.956472,0.029261,0.078126,0.011395,0.254585,0.000000,5.166667,2.049676,1.092412,0.000000
360630243032,0.581794,0.075598,0.216960,0.012797,0.111398,0.000000,3.166667,2.197988,1.466803,0.000000
360070143012,0.802748,0.246363,0.574848,3.218809,0.303057,0.000000,8.166667,7.899193,14.398414,0.000000
360810259002,0.744217,28.056812,80.037065,57.240906,0.833942,0.506986,17.333333,25.424383,133.630585,0.000283
360810166002,0.560604,19.796932,69.444270,6.288958,0.166082,0.029180,14.333333,29.468186,113.378400,0.000016
...,...,...,...,...,...,...,...,...,...,...
361190050012,0.861077,4.482020,11.395580,11.232268,0.992783,0.004891,18.166667,28.311987,175.644800,0.000003
361190051003,0.750954,9.200238,15.417065,4.283444,0.434856,0.005413,16.000000,25.306068,165.947324,0.000003
361190053002,0.838092,8.896390,23.193360,7.910393,0.508646,0.018760,17.666667,29.438398,118.500906,0.000010


In [7]:
## Option 2: Removing Rows based on Constraints

# Create a boolean mask for each constraint
mask_D2B_E8MIX = (rqs_df['D2B_E8MIX'] >= 0) & (rqs_df['D2B_E8MIX'] <= 1)
mask_D5DR = (rqs_df['D5DR'] >= 0) & (rqs_df['D5DR'] <= 1)
mask_D5DRI = (rqs_df['D5DRI'] >= 0) & (rqs_df['D5DRI'] <= 1)
mask_D1A = rqs_df['D1A'] >= 0
mask_D1B = rqs_df['D1B'] >= 0
mask_D1C = rqs_df['D1C'] >= 0
mask_D2R_JOBPOP = rqs_df['D2R_JOBPOP'] >= 0
mask_D3A = rqs_df['D3A'] >= 0
mask_D3B = rqs_df['D3B'] >= 0
mask_NatWalkInd = (rqs_df['NatWalkInd'] >= 0) & (rqs_df['NatWalkInd'] <= 100)

# Combine all masks using logical AND
combined_mask = mask_D2B_E8MIX & mask_D5DR & mask_D5DRI & mask_D1A & mask_D1B & mask_D1C & mask_D2R_JOBPOP & mask_D3A & mask_D3B & mask_NatWalkInd

# Apply the combined mask to filter the DataFrame
filtered_rqs_df = rqs_df[combined_mask]
filtered_rqs_df

,D2B_E8MIX,D1A,D1B,D1C,D2R_JOBPOP,D5DRI,NatWalkInd,D3A,D3B,D5DR
GEOID20,,,,,,,,,,
360810259002,0.744217,28.056812,80.037065,57.240906,0.833942,0.506986,17.333333,25.424383,133.630585,0.000283
360810166002,0.560604,19.796932,69.444270,6.288958,0.166082,0.029180,14.333333,29.468186,113.378400,0.000016
360811227021,0.000000,53.666938,131.824297,0.000000,0.000000,0.154780,9.666667,15.441301,59.517861,0.000086
360810283005,0.890368,137.710892,286.833683,18.544335,0.121452,0.535719,10.833333,21.479258,0.000000,0.000299
360811483004,0.803865,4.776298,12.819061,2.221196,0.295367,0.082963,15.333333,27.884708,205.067423,0.000046
...,...,...,...,...,...,...,...,...,...,...
361190050012,0.861077,4.482020,11.395580,11.232268,0.992783,0.004891,18.166667,28.311987,175.644800,0.000003
361190051003,0.750954,9.200238,15.417065,4.283444,0.434856,0.005413,16.000000,25.306068,165.947324,0.000003
361190053002,0.838092,8.896390,23.193360,7.910393,0.508646,0.018760,17.666667,29.438398,118.500906,0.000010




---



## Sub-Question 1:
### Does higher transit service frequency and job accessibility via transit correlate with reduced car ownership and car usage?


#### Variables:

* Transit Frequency:
  * D4C: Aggregate frequency of transit service within 0.25 miles of CBG boundary per hour during evening peak period
  * D4E: Aggregate frequency of transit service per capita
* Job Accessibility:
  * D5BR: Jobs within 45-minute transit commute, distance decay (walk network travel time, GTFS schedules) weighted
  * D5DR: Proportional Accessibility of Regional Destinations by Transit (Employment accessibility expressed as a ratio of total MSA accessibility)
* Car Ownership & Data:
  * AutoOwn0: Number of households in CBG that own zero automobiles
  * AutoOwn1: Number of households in CBG that own one automobile
  * AutoOwn2p: Number of households in CBG that own two or more automobiles
  * VMT_tot_avg: Weighted average daily Vehicle Miles Traveled (VMT) per worker




In [8]:
q1_df = ny_smart_location_df[['D4C', 'D4E', 'D5BR', 'D5DR', 'AutoOwn0', 'AutoOwn1', 'AutoOwn2p', 'VMT_tot_avg']]
q1_df.head()

,D4C,D4E,D5BR,D5DR,AutoOwn0,AutoOwn1,AutoOwn2p,VMT_tot_avg
GEOID20,,,,,,,,
360150102001,-99999.00,-99999.000000,-99999.0,-99999.000000,0,30,243,19.519636
360630243032,-99999.00,-99999.000000,-99999.0,-99999.000000,27,165,446,17.468909
360070143012,-99999.00,-99999.000000,-99999.0,-99999.000000,35,105,151,20.926535
360810259002,113.33,0.088678,3530637.0,0.000283,230,155,20,11.177436
360810166002,27.67,0.017647,203212.0,0.000016,120,203,124,11.177436


In [9]:
## Option 1: Altering Rows based on Constraints
op1 = q1_df.copy()

# Enforce non-negative constraint for D4C, D4E, D5BR, AutoOwn0, AutoOwn1, AutoOwn2p and VMT_tot_avg
op1['D4C'] = op1['D4C'].clip(lower=0)
op1['D4E'] = op1['D4E'].clip(lower=0)
op1['D5BR'] = op1['D5BR'].clip(lower=0)
op1['AutoOwn0'] = op1['AutoOwn0'].clip(lower=0)
op1['AutoOwn1'] = op1['AutoOwn1'].clip(lower=0)
op1['AutoOwn2p'] = op1['AutoOwn2p'].clip(lower=0)
op1['VMT_tot_avg'] = op1['VMT_tot_avg'].clip(lower=0)

# Enforce constraint for D5DR (between 0 and 1)
op1['D5DR'] = np.clip(op1['D5DR'], 0, 1)

op1

,D4C,D4E,D5BR,D5DR,AutoOwn0,AutoOwn1,AutoOwn2p,VMT_tot_avg
GEOID20,,,,,,,,
360150102001,0.00,0.000000,0.0,0.000000,0,30,243,19.519636
360630243032,0.00,0.000000,0.0,0.000000,27,165,446,17.468909
360070143012,0.00,0.000000,0.0,0.000000,35,105,151,20.926535
360810259002,113.33,0.088678,3530637.0,0.000283,230,155,20,11.177436
360810166002,27.67,0.017647,203212.0,0.000016,120,203,124,11.177436
...,...,...,...,...,...,...,...,...
361190050012,31.67,0.025215,34059.0,0.000003,44,205,227,11.177436
361190051003,14.00,0.015135,37696.0,0.000003,93,230,200,11.177436
361190053002,15.33,0.007406,130646.0,0.000010,174,256,301,11.177436


In [10]:
## Option 2: Removing Rows based on Constraints

# Create a boolean mask for each constraint
mask_D4C = q1_df['D4C'] >= 0
mask_D4E = q1_df['D4E'] >= 0
mask_D5BR = q1_df['D5BR'] >= 0
mask_AutoOwn0 = q1_df['AutoOwn0'] >= 0
mask_AutoOwn1 = q1_df['AutoOwn1'] >= 0
mask_AutoOwn2p = q1_df['AutoOwn2p'] >= 0
mask_VMT_tot_avg = q1_df['VMT_tot_avg'] >= 0
mask_D5DR = (q1_df['D5DR'] >= 0) & (q1_df['D5DR'] <= 1)

# Combine all masks using logical AND
combined_mask = mask_D4C & mask_D4E & mask_D5BR & mask_AutoOwn0 & mask_AutoOwn1 & mask_AutoOwn2p & mask_VMT_tot_avg & mask_D5DR

# Apply the combined mask to filter the DataFrame
filtered_q1_df = q1_df[combined_mask]

filtered_q1_df

,D4C,D4E,D5BR,D5DR,AutoOwn0,AutoOwn1,AutoOwn2p,VMT_tot_avg
GEOID20,,,,,,,,
360810259002,113.33,0.088678,3530637.0,0.000283,230,155,20,11.177436
360810166002,27.67,0.017647,203212.0,0.000016,120,203,124,11.177436
360811227021,37.00,0.015658,1077886.0,0.000086,803,129,30,11.177436
360810283005,85.67,0.032774,3730728.0,0.000299,634,502,45,11.177436
360811483004,8.33,0.009434,577755.0,0.000046,12,123,194,11.177436
...,...,...,...,...,...,...,...,...
361190050012,31.67,0.025215,34059.0,0.000003,44,205,227,11.177436
361190051003,14.00,0.015135,37696.0,0.000003,93,230,200,11.177436
361190053002,15.33,0.007406,130646.0,0.000010,174,256,301,11.177436




---



## Sub-Question 2:
### Do residents in high income-entropy neighborhoods have greater access to jobs via transit (greater number of regional jobs accessible by transit) than those in low-entropy neighborhoods?

Variables:


*   Wage:
  * W_P_Lowwage: Percent low wage workers
  * W_P_Medwage: Percent medium wage workers
  * W_P_Highwage: Percent of high wage workers
* Job Accessibility:
  * D5DR: Proportional Accessibility of Regional Destinations by Transit (Employment accessibility expressed as a ratio of total MSA accessibility)
  

In [11]:
q2_df = ny_smart_location_df[['W_P_Lowwage', 'W_P_Medwage', 'W_P_Highwage', 'D5DR']]
q2_df.head()

,W_P_Lowwage,W_P_Medwage,W_P_Highwage,D5DR
GEOID20,,,,
360150102001,0.652542,0.220339,0.127119,-99999.000000
360630243032,0.518519,0.342593,0.138889,-99999.000000
360070143012,0.233561,0.414256,0.352183,-99999.000000
360810259002,0.284464,0.269147,0.446389,0.000283
360810166002,0.267606,0.345070,0.387324,0.000016


In [12]:
## Option 1: Altering Rows based on Constraints
op1 = q2_df.copy()

# Enforce constraints for W_P_Lowwage, W_P_Medwage, W_P_Highwage, and D5DR (between 0 and 1)
op1['W_P_Lowwage'] = np.clip(op1['W_P_Lowwage'], 0, 1)
op1['W_P_Medwage'] = np.clip(op1['W_P_Medwage'], 0, 1)
op1['W_P_Highwage'] = np.clip(op1['W_P_Highwage'], 0, 1)
op1['D5DR'] = np.clip(op1['D5DR'], 0, 1)

op1

,W_P_Lowwage,W_P_Medwage,W_P_Highwage,D5DR
GEOID20,,,,
360150102001,0.652542,0.220339,0.127119,0.000000
360630243032,0.518519,0.342593,0.138889,0.000000
360070143012,0.233561,0.414256,0.352183,0.000000
360810259002,0.284464,0.269147,0.446389,0.000283
360810166002,0.267606,0.345070,0.387324,0.000016
...,...,...,...,...
361190050012,0.187399,0.333603,0.478998,0.000003
361190051003,0.245136,0.342412,0.412451,0.000003
361190053002,0.270538,0.337110,0.392351,0.000010


In [13]:
## Option 2: Removing Rows based on Constraints

# Create a boolean mask for each constraint
mask_W_P_Lowwage = (q2_df['W_P_Lowwage'] >= 0) & (q2_df['W_P_Lowwage'] <= 1)
mask_W_P_Medwage = (q2_df['W_P_Medwage'] >= 0) & (q2_df['W_P_Medwage'] <= 1)
mask_W_P_Highwage = (q2_df['W_P_Highwage'] >= 0) & (q2_df['W_P_Highwage'] <= 1)
mask_D5DR = (q2_df['D5DR'] >= 0) & (q2_df['D5DR'] <= 1)

# Combine all masks using logical AND
combined_mask = mask_W_P_Lowwage & mask_W_P_Medwage & mask_W_P_Highwage & mask_D5DR

# Apply the combined mask to filter the DataFrame
filtered_q2_df = q2_df[combined_mask]
filtered_q2_df

,W_P_Lowwage,W_P_Medwage,W_P_Highwage,D5DR
GEOID20,,,,
360810259002,0.284464,0.269147,0.446389,0.000283
360810166002,0.267606,0.345070,0.387324,0.000016
360811227021,0.211386,0.317605,0.470971,0.000086
360810283005,0.366864,0.355030,0.278107,0.000299
360811483004,0.124183,0.294118,0.581699,0.000046
...,...,...,...,...
361190050012,0.187399,0.333603,0.478998,0.000003
361190051003,0.245136,0.342412,0.412451,0.000003
361190053002,0.270538,0.337110,0.392351,0.000010




---



## Sub-Question 3:
### Does higher pedestrian-oriented urban design correlate with reduced greenhouse gas (GHG) emissions?




Variables:


* Pedestrian-Oriented Urban Design
  * D3APO: Network density in terms of facility miles of pedestrian-oriented links per square mile
  * D3BPO3: Intersection density in terms of pedestrian-oriented intersections having three legs per square mile
  * D3BPO4: Intersection density in terms of pedestrian-oriented intersections having four or more legs per square mile
* GHG
  * Workers_1: Number of workers
  * GHG_per_worker: Estimated average daily GHG generated by a worker in a workplace block group


In [14]:
q3_df = ny_smart_location_df[['D3APO', 'D3BPO3', 'D3BPO4', 'Workers_1', 'GHG_per_worker']]
q3_df.head()

,D3APO,D3BPO3,D3BPO4,Workers_1,GHG_per_worker
GEOID20,,,,,
360150102001,1.379314,1.112498,0.061805,290.0,22.616242
360630243032,0.734747,0.379176,0.000000,813.0,20.666189
360070143012,5.815968,20.787700,0.533018,269.0,22.933923
360810259002,21.546514,40.081159,0.000000,620.0,2.836972
360810166002,15.533187,0.000000,56.689200,573.0,5.299860


In [15]:
## Option 1: Altering Rows based on Constraints
op3 = q3_df.copy()

# Enforce non-negative constraint for all specified columns
for column in ['D3APO', 'D3BPO3', 'D3BPO4', 'Workers_1', 'GHG_per_worker']:
    op3[column] = op3[column].clip(lower=0)

op3

,D3APO,D3BPO3,D3BPO4,Workers_1,GHG_per_worker
GEOID20,,,,,
360150102001,1.379314,1.112498,0.061805,290.0,22.616242
360630243032,0.734747,0.379176,0.000000,813.0,20.666189
360070143012,5.815968,20.787700,0.533018,269.0,22.933923
360810259002,21.546514,40.081159,0.000000,620.0,2.836972
360810166002,15.533187,0.000000,56.689200,573.0,5.299860
...,...,...,...,...,...
361190050012,27.264082,134.387758,64.506124,718.0,12.673701
361190051003,24.113311,211.748532,10.587427,557.0,10.486647
361190053002,21.346148,42.651796,42.651796,922.0,9.469089


In [16]:
## Option 2: Removing Rows based on Constraints

# Create a boolean mask for each constraint
mask_D3APO = q3_df['D3APO'] >= 0
mask_D3BPO3 = q3_df['D3BPO3'] >= 0
mask_D3BPO4 = q3_df['D3BPO4'] >= 0
mask_Workers_1 = q3_df['Workers_1'] >= 0
mask_GHG_per_worker = q3_df['GHG_per_worker'] >= 0

# Combine all masks using logical AND
combined_mask = mask_D3APO & mask_D3BPO3 & mask_D3BPO4 & mask_Workers_1 & mask_GHG_per_worker

# Apply the combined mask to filter the DataFrame
filtered_q3_df = q3_df[combined_mask]

filtered_q3_df

,D3APO,D3BPO3,D3BPO4,Workers_1,GHG_per_worker
GEOID20,,,,,
360150102001,1.379314,1.112498,0.061805,290.0,22.616242
360630243032,0.734747,0.379176,0.000000,813.0,20.666189
360070143012,5.815968,20.787700,0.533018,269.0,22.933923
360810259002,21.546514,40.081159,0.000000,620.0,2.836972
360810166002,15.533187,0.000000,56.689200,573.0,5.299860
...,...,...,...,...,...
361190050012,27.264082,134.387758,64.506124,718.0,12.673701
361190051003,24.113311,211.748532,10.587427,557.0,10.486647
361190053002,21.346148,42.651796,42.651796,922.0,9.469089
